## Professional Athlete Classifier

#### Connect To Workspace

In [5]:
from azureml.core import Workspace

workspace = Workspace.from_config()

#### Data Prep

Use Azure Cognitive Services to search internet for photos of NBA players and then upload to blob storage

- TODO: Add instructions on how to set up cognitive services
- TODO: Add packages o install (cognitive service sdk)

In [6]:
datastore = workspace.get_default_datastore()

In [12]:
config_file_name = 'blobfuse_connection.cfg'

config_params = 'accountName {}\naccountKey {}\ncontainerName {}'.format(
    datastore.account_name, 
    datastore.account_key, 
    datastore.container_name)

config_file = open(config_file_name, 'w')
config_file.writelines(config_params)
config_file.close()

In [14]:
blobstore_dir = 'blobstore'

In [ ]:
import os
if not os.path.exists(blobstore_dir):
    os.makedirs(blobstore_dir)

In [15]:
!sudo blobfuse {blobstore_dir} --tmp-path=/mnt/resource/blobfusetmp  --config-file={config_file_name} -o attr_timeout=240 -o entry_timeout=240 -o negative_timeout=120

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [17]:
data_dir = 'nba-players'
!sudo mkdir {blobstore_dir}/{data_dir}

In [18]:
import os
import sys

!sudo {sys.executable} cognitive-service-web-scraper.py --root_dir {blobstore_dir}/{data_dir} --image_dim 400

Writing image 0 for Lebron James...
Writing image 1 for Lebron James...
Writing image 2 for Lebron James...
Writing image 3 for Lebron James...
Writing image 4 for Lebron James...
Writing image 5 for Lebron James...
Writing image 6 for Lebron James...
Writing image 7 for Lebron James...
Writing image 8 for Lebron James...
Writing image 9 for Lebron James...
Writing image 10 for Lebron James...
Writing image 11 for Lebron James...
Writing image 12 for Lebron James...
Writing image 13 for Lebron James...
Writing image 14 for Lebron James...
Writing image 15 for Lebron James...
Writing image 16 for Lebron James...
Writing image 17 for Lebron James...
Writing image 18 for Lebron James...
Writing image 19 for Lebron James...
Writing image 20 for Lebron James...
Writing image 21 for Lebron James...
Writing image 22 for Lebron James...
Writing image 23 for Lebron James...
Writing image 24 for Lebron James...
Writing image 25 for Lebron James...
Writing image 26 for Lebron James...
Writing ima

Writing image 210 for Lebron James...
Writing image 211 for Lebron James...
Writing image 212 for Lebron James...
Writing image 213 for Lebron James...
Writing image 214 for Lebron James...
Writing image 215 for Lebron James...
Writing image 216 for Lebron James...
Writing image 217 for Lebron James...
Writing image 218 for Lebron James...
Writing image 219 for Lebron James...
Writing image 220 for Lebron James...
Writing image 221 for Lebron James...
Writing image 222 for Lebron James...
Writing image 223 for Lebron James...
Writing image 224 for Lebron James...
Writing image 225 for Lebron James...
Writing image 226 for Lebron James...
Writing image 227 for Lebron James...
Writing image 228 for Lebron James...
Writing image 229 for Lebron James...
Writing image 230 for Lebron James...
Writing image 231 for Lebron James...
Writing image 232 for Lebron James...
Writing image 233 for Lebron James...
Writing image 234 for Lebron James...
Writing image 235 for Lebron James...
Writing imag

Writing image 420 for Lebron James...
Writing image 421 for Lebron James...
Writing image 422 for Lebron James...
Writing image 423 for Lebron James...
Skipping blobstore/nba-players/valid/Lebron_James/423.jpg due to download error:
Writing image 423 for Lebron James...
Writing image 424 for Lebron James...
Writing image 425 for Lebron James...
Writing image 426 for Lebron James...
Writing image 427 for Lebron James...
Writing image 428 for Lebron James...
Writing image 429 for Lebron James...
Writing image 430 for Lebron James...
Writing image 431 for Lebron James...
Writing image 432 for Lebron James...
Writing image 433 for Lebron James...
Writing image 434 for Lebron James...
Writing image 435 for Lebron James...
Writing image 436 for Lebron James...
Writing image 437 for Lebron James...
Writing image 438 for Lebron James...
Writing image 439 for Lebron James...
Writing image 440 for Lebron James...
Writing image 441 for Lebron James...
Writing image 442 for Lebron James...
Writing

Writing image 128 for Stephen Curry...
Writing image 129 for Stephen Curry...
Writing image 130 for Stephen Curry...
Writing image 131 for Stephen Curry...
Writing image 132 for Stephen Curry...
Writing image 133 for Stephen Curry...
Writing image 134 for Stephen Curry...
Writing image 135 for Stephen Curry...
Writing image 136 for Stephen Curry...
Writing image 137 for Stephen Curry...
Writing image 138 for Stephen Curry...
Writing image 139 for Stephen Curry...
Writing image 140 for Stephen Curry...
Writing image 141 for Stephen Curry...
Writing image 142 for Stephen Curry...
Writing image 143 for Stephen Curry...
Writing image 144 for Stephen Curry...
Writing image 145 for Stephen Curry...
Writing image 146 for Stephen Curry...
Writing image 147 for Stephen Curry...
Writing image 148 for Stephen Curry...
Writing image 149 for Stephen Curry...
Writing image 150 for Stephen Curry...
Writing image 151 for Stephen Curry...
Writing image 152 for Stephen Curry...
Writing image 153 for Ste

Writing image 329 for Stephen Curry...
Writing image 330 for Stephen Curry...
Writing image 331 for Stephen Curry...
Writing image 332 for Stephen Curry...
Writing image 333 for Stephen Curry...
Writing image 334 for Stephen Curry...
Writing image 335 for Stephen Curry...
Writing image 336 for Stephen Curry...
Writing image 337 for Stephen Curry...
Writing image 338 for Stephen Curry...
Writing image 339 for Stephen Curry...
Writing image 340 for Stephen Curry...
Writing image 341 for Stephen Curry...
Writing image 342 for Stephen Curry...
Writing image 343 for Stephen Curry...
Writing image 344 for Stephen Curry...
Writing image 345 for Stephen Curry...
Writing image 346 for Stephen Curry...
Writing image 347 for Stephen Curry...
Writing image 348 for Stephen Curry...
Writing image 349 for Stephen Curry...
Writing image 350 for Stephen Curry...
Writing image 351 for Stephen Curry...
Writing image 352 for Stephen Curry...
Writing image 353 for Stephen Curry...
Writing image 354 for Ste

...

Cropping image: blobstore/nba-players/train/Stephen_Curry/81.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/338.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/182.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/203.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/181.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/45.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/0.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/132.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/340.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/29.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/113.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/139.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/187.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/150.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/34.jpg
Cropping image: 

Cropping image: blobstore/nba-players/train/Stephen_Curry/1.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/248.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/97.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/274.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/307.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/329.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/75.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/328.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/111.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/298.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/229.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/211.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/215.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/228.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/308.jpg
Cropping image

Cropping image: blobstore/nba-players/train/Lebron_James/178.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/179.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/18.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/180.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/181.jpg
Removing corrupted file: blobstore/nba-players/train/Lebron_James/181.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/182.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/183.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/184.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/185.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/186.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/187.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/188.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/189.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/19.jpg
Cropping image: bl

Cropping image: blobstore/nba-players/train/Lebron_James/288.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/289.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/29.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/290.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/291.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/292.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/293.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/294.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/295.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/296.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/297.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/298.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/299.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/3.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/30.jpg
Cropping image: blobstore/nba

Cropping image: blobstore/nba-players/train/Lebron_James/86.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/87.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/88.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/89.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/9.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/90.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/91.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/92.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/93.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/94.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/95.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/96.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/97.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/98.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/99.jpg


In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir=data_dir, target_path='athletes')

## Train

- Requirement packages to add: imutils, opencv-python